In [ ]:
import pandas as pd
from datasets import Dataset
from transformers import T5Tokenizer, T5ForConditionalGeneration, TrainingArguments, Trainer

# Load data
df = pd.read_csv("/content/drive/MyDrive/fullcommand2.csv")

# Keep only the columns we need
df = df[['question', 'command']].dropna()

# Rename for HuggingFace format
df = df.rename(columns={'question': 'input_text', 'command': 'target_text'})

# Convert to HuggingFace Dataset
dataset = Dataset.from_pandas(df)

# Split into train/test
dataset = dataset.train_test_split(test_size=0.1)

# Load tokenizer
tokenizer = T5Tokenizer.from_pretrained('t5-base')

# Tokenization function
def preprocess(example):
    input_enc = tokenizer(example['input_text'], padding='max_length', truncation=True, max_length=64)
    target_enc = tokenizer(example['target_text'], padding='max_length', truncation=True, max_length=64)
    input_enc['labels'] = target_enc['input_ids']
    return input_enc

# Tokenize dataset
tokenized = dataset.map(preprocess, batched=True)

# Load model
model = T5ForConditionalGeneration.from_pretrained('t5-base')

# Training arguments
training_args = TrainingArguments(
    output_dir="./t5_kube_model",
    per_device_train_batch_size=4,
    per_device_eval_batch_size=4,
    learning_rate=3e-4,
    num_train_epochs=4,
    eval_strategy="epoch",
    logging_dir='./logs',
    save_strategy="epoch",
    weight_decay=0.01,
    push_to_hub=False,
    report_to="none",
)

# Trainer
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=tokenized["train"],
    eval_dataset=tokenized["test"],
    tokenizer=tokenizer
)

# Train!
trainer.train()

/usr/local/lib/python3.12/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


spiece.model:   0%|          | 0.00/792k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.39M [00:00<?, ?B/s]

config.json:   0%|          | 0.00/1.21k [00:00<?, ?B/s]

You are using the default legacy behaviour of the <class 'transformers.models.t5.tokenization_t5.T5Tokenizer'>. This is expected, and simply means that the `legacy` (previous) behavior will be used so nothing changes for you. If you want to use the new behaviour, set `legacy=False`. This should only be set if you understand what it means, and thoroughly read the reason why this was added as explained in https://github.com/huggingface/transformers/pull/24565


...

Map:   0%|          | 0/3489 [00:00<?, ? examples/s]

model.safetensors:   0%|          | 0.00/892M [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/147 [00:00<?, ?B/s]

/tmp/ipython-input-3932082829.py:52: FutureWarning: `tokenizer` is deprecated and will be removed in version 5.0.0 for `Trainer.__init__`. Use `processing_class` instead.
  trainer = Trainer(


Epoch,Training Loss,Validation Loss


In [ ]:
!cp -r /content/t5_kube_model/checkpoint-23547 /content/drive/MyDrive/

cp: cannot stat '/content/t5_kube_model/checkpoint-23547': No such file or directory


In [3]:
from transformers import T5ForConditionalGeneration, T5Tokenizer

# model = T5ForConditionalGeneration.from_pretrained("/content/t5_kube_model/checkpoint-31396")
model = T5ForConditionalGeneration.from_pretrained("/content/drive/MyDrive/checkpoint-23547")
tokenizer = T5Tokenizer.from_pretrained("t5-base")

def predict(text):
    inputs = tokenizer(text, return_tensors='pt', truncation=True, padding=True)
    output_ids = model.generate(inputs.input_ids, max_length=64)
    return tokenizer.decode(output_ids[0], skip_special_tokens=True)

# Example
print(predict("Create TLS certificates|"))


Asking to truncate to max_length but no maximum length is provided and the model has no predefined maximum length. Default to no truncation.


kubectl create certificate tls --class=default  --rule="foo.com/*=svc:8081,tls=secret1"


In [ ]:
print(predict("Create a deployment of nginx with 3 replicas"))

kubectl create deployment nginx --image=nginx --replicas=3


In [4]:
print(predict("Print the API resources available in the 'hugging' API group"))

kubectl api-resources --api-group=hugging


In [ ]:
print(predict("Get the documentation of a specific field of a resource"))

kubectl explain resource=pods


In [ ]:
from transformers import T5ForConditionalGeneration, T5Tokenizer

# model = T5ForConditionalGeneration.from_pretrained("/content/t5_kube_model/checkpoint-31396")
model = T5ForConditionalGeneration.from_pretrained("/content/drive/MyDrive/checkpoint-23547")
tokenizer = T5Tokenizer.from_pretrained("t5-base")

def predict(text):
    inputs = tokenizer(text, return_tensors='pt', truncation=True, padding=True)
    output_ids = model.generate(inputs.input_ids, max_length=64)
    return tokenizer.decode(output_ids[0], skip_special_tokens=True)

# Example
print(predict("Create TLS certificates|"))


/usr/local/lib/python3.12/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


spiece.model:   0%|          | 0.00/792k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.39M [00:00<?, ?B/s]

config.json:   0%|          | 0.00/1.21k [00:00<?, ?B/s]

You are using the default legacy behaviour of the <class 'transformers.models.t5.tokenization_t5.T5Tokenizer'>. This is expected, and simply means that the `legacy` (previous) behavior will be used so nothing changes for you. If you want to use the new behaviour, set `legacy=False`. This should only be set if you understand what it means, and thoroughly read the reason why this was added as explained in https://github.com/huggingface/transformers/pull/24565
Asking to truncate to max_length but no maximum length is provided and the model has no predefined maximum length. Default to no truncation.


kubectl create certificate tls --class=default  --rule="foo.com/*=svc:8081,tls=secret1"


# API ngrok

In [5]:
!pip install -q flask-ngrok

In [9]:
!pip install pyngrok flask

In [ ]:
!export NGROK_AUTHTOKEN=

In [ ]:
!ngrok config add-authtoken 

Authtoken saved to configuration file: /root/.config/ngrok/ngrok.yml


In [6]:
from flask import Flask, request, jsonify
from flask_ngrok import run_with_ngrok
from transformers import T5ForConditionalGeneration, T5Tokenizer

app = Flask(__name__)
run_with_ngrok(app)  # exposes it via ngrok public URL

# Load model and tokenizer
model = T5ForConditionalGeneration.from_pretrained("/content/drive/MyDrive/checkpoint-23547")
tokenizer = T5Tokenizer.from_pretrained("t5-base")

def predict(text):
    inputs = tokenizer(text, return_tensors='pt', truncation=True, padding=True)
    output_ids = model.generate(inputs.input_ids, max_length=64)
    return tokenizer.decode(output_ids[0], skip_special_tokens=True)

@app.route('/predict', methods=['GET'])
def predict_endpoint():
    query = request.args.get('q', '')
    if not query:
        return jsonify({'error': 'Missing query parameter "q"'}), 400
    result = predict(query)
    return jsonify({'input': query, 'output': result})

if __name__ == '__main__':
    app.run()


 * Serving Flask app '__main__'
 * Debug mode: off


INFO:werkzeug:WARNING: This is a development server. Do not use it in a production deployment. Use a production WSGI server instead.
 * Running on http://127.0.0.1:5000
INFO:werkzeug:Press CTRL+C to quit
Exception in thread Thread-6:
Traceback (most recent call last):
  File "/usr/local/lib/python3.12/dist-packages/urllib3/connection.py", line 198, in _new_conn
    sock = connection.create_connection(
           ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "/usr/local/lib/python3.12/dist-packages/urllib3/util/connection.py", line 85, in create_connection
    raise err
  File "/usr/local/lib/python3.12/dist-packages/urllib3/util/connection.py", line 73, in create_connection
    sock.connect(sa)
ConnectionRefusedError: [Errno 111] Connection refused

The above exception was the direct cause of the following exception:

Traceback (most recent call last):
  File "/usr/local/lib/python3.12/dist-packages/urllib3/connectionpool.py", line 787, in urlopen
    response = self._make_request(
            

In [ ]:
from flask import Flask, request, jsonify
from pyngrok import ngrok
from transformers import T5ForConditionalGeneration, T5Tokenizer

# Load model and tokenizer
model = T5ForConditionalGeneration.from_pretrained("/content/drive/MyDrive/checkpoint-23547")
tokenizer = T5Tokenizer.from_pretrained("t5-base")

def predict(text):
    inputs = tokenizer(text, return_tensors='pt', truncation=True, padding=True)
    output_ids = model.generate(inputs.input_ids, max_length=64)
    return tokenizer.decode(output_ids[0], skip_special_tokens=True)

app = Flask(__name__)

@app.route('/predict', methods=['GET'])
def predict_endpoint():
    query = request.args.get('q', '')
    if not query:
        return jsonify({'error': 'Missing query parameter "q"'}), 400
    result = predict(query)
    return jsonify({'input': query, 'output': result})

# ---- Start ngrok tunnel manually ----
public_url = ngrok.connect(5000).public_url
print(f"✅ Public API URL: {public_url}/predict?q=Create+TLS+certificates")

# ---- Run Flask normally ----
app.run(port=5000)


✅ Public API URL: https://labile-overdevotedly-marcell.ngrok-free.dev/predict?q=Create+TLS+certificates
 * Serving Flask app '__main__'
 * Debug mode: off


INFO:werkzeug:WARNING: This is a development server. Do not use it in a production deployment. Use a production WSGI server instead.
 * Running on http://127.0.0.1:5000
INFO:werkzeug:Press CTRL+C to quit
Asking to truncate to max_length but no maximum length is provided and the model has no predefined maximum length. Default to no truncation.
INFO:werkzeug:127.0.0.1 - - [29/Oct/2025 13:33:04] "GET /predict?q=Create+TLS+certificates HTTP/1.1" 200 -
INFO:werkzeug:127.0.0.1 - - [29/Oct/2025 13:33:05] "GET /favicon.ico HTTP/1.1" 404 -
INFO:werkzeug:127.0.0.1 - - [29/Oct/2025 13:33:11] "GET /predict?q=Create+TLS+certificates HTTP/1.1" 200 -
INFO:werkzeug:127.0.0.1 - - [29/Oct/2025 13:33:14] "GET /predict?q=Create+TLS+certificates HTTP/1.1" 200 -
INFO:werkzeug:127.0.0.1 - - [29/Oct/2025 13:36:59] "GET /predict?q=Create%20TLS%20certificate HTTP/1.1" 200 -
INFO:werkzeug:127.0.0.1 - - [29/Oct/2025 13:37:26] "GET /predict?q=create%20nginx%20deployment HTTP/1.1" 200 -
INFO:werkzeug:127.0.0.1 - - 

# Upload

In [7]:
!cp -r /content/drive/MyDrive/checkpoint-23547 /content/model

In [ ]:
!pip -q install huggingface_hub
from huggingface_hub import HfApi, create_repo
from huggingface_hub import hf_hub_download
from huggingface_hub import login

login(token="")
# --- config ---
repo_id       = "tarun122/t5-cpt-2"        # e.g. "alice/my-model"
folder_path   = "/content/model"  # the folder you want to upload
repo_type     = "model"                   # or "dataset" / "space"
private       = False                     # True if you want it private
commit_msg    = "Upload folder"
path_in_repo  = ""                        # optional subdir inside repo, e.g. "data/"

# 1) Make sure the repo exists (idempotent)
create_repo(repo_id, repo_type=repo_type, private=private, exist_ok=True)

# 2) Upload the entire folder
api = HfApi()
api.upload_folder(
    repo_id=repo_id,
    repo_type=repo_type,
    folder_path=folder_path,
    path_in_repo=path_in_repo,        # "" to upload at repo root
    commit_message=commit_msg,
    # token=None                      # optional; otherwise uses HF_TOKEN env or cached login
)
print("Done!")

Processing Files (0 / 0)      : |          |  0.00B /  0.00B            

New Data Upload               : |          |  0.00B /  0.00B            

  ...point-23547/rng_state.pth:  78%|#######7  | 11.0kB / 14.2kB            

  ...kpoint-23547/spiece.model: 100%|##########|  792kB /  792kB            

  ...t-23547/model.safetensors:   0%|          |  553kB /  892MB            

  ...kpoint-23547/optimizer.pt:   0%|          | 38.2kB / 1.78GB            

  ...kpoint-23547/scheduler.pt:   6%|6         |  69.0B / 1.06kB            

  ...t-23547/training_args.bin:   7%|6         |   342B / 5.24kB            

Done!


Fri Aug  1 19:14:07 2025       
+-----------------------------------------------------------------------------------------+
| NVIDIA-SMI 550.54.15              Driver Version: 550.54.15      CUDA Version: 12.4     |
|-----------------------------------------+------------------------+----------------------+
| GPU  Name                 Persistence-M | Bus-Id          Disp.A | Volatile Uncorr. ECC |
| Fan  Temp   Perf          Pwr:Usage/Cap |           Memory-Usage | GPU-Util  Compute M. |
|                                         |                        |               MIG M. |
|=========================================+========================+======================|
|   0  Tesla T4                       Off |   00000000:00:04.0 Off |                    0 |
| N/A   44C    P8              9W /   70W |       2MiB /  15360MiB |      0%      Default |
|                                         |                        |                  N/A |
+-----------------------------------------+-----